In [1]:
import numpy as np
import random as random
import matplotlib.pyplot as plt
from PIL import Image as im
import time

In [2]:
###functions
def Mscatter(Mie):
    #returns 1 if a scattering event occurs
    p=random.random()
    if p > (1-Mie):
        return 1
    else:
        return 0
    
def Rscatter(Ryl,Wvl):
    #returns 1 if a scattering event occurs
    p=random.random()
    if (p > (1-Ryl*Wvl**-4)):
        return 1
    else:
        return 0

def muM(g):
    #mu parameter for Mie scattering, mu=cos(zenith scattering angle) 
    p=random.random()
    return (1/(2*g))*(1+g**2-((1-g**2)/(1+2*g*p-g))**2)
    
def muR():
    #mu parameter for Rayleigh scattering
    Rmu=random.random()
    q=4-8*Rmu
    a=(q**2+4)**0.5
    return ((a-q)**(3**-1)-(a+q)**(3**-1))*(1/2)**(3**-1)

#rotation matricies
def rotate(angle, axis):
    #accept radians
    return np.cos(angle)*np.identity(3)+np.sin(angle)*np.array([[0,-axis[2,0],axis[1,0]],[axis[2,0],0,-axis[0,0]],[-axis[1,0],axis[0,0],0]])+(1-np.cos(angle))*np.outer(axis,axis)

#normalize a vector
def normalize(v):
    return v/(np.sum(v**2)**0.5)

In [5]:
###simulaiton and output settings
#start position vector (unused)
#r0=np.array([0,0,6371000])
#planet radii from WGS-84
rp=6356752
re=6378137
#Rp=6371000
#atmosphere height
Ha=8400
#Ra=Ha+Rp
#color array (3 colors, wavelength in m) R, G, B
ll=np.array([630,532,465])*1E-9
#max steps until ray dies
lifespan=250
#light propogation step size (25km max light path length)
lstep=25000/lifespan
#rayleigh scattering strength (fraction is based on noon transmittance)
SCr=(1-(1040/1353)**(lstep/Ha))*ll[1]**4
SCr=SCr*lstep/Ha #per step
#mie scattering strength (fraction arbitrary)
#MIE REFERENCE TABLE
#Condition   Fraction/1000
#Very Clean  990
#Average     905-861
#Very Hazy   670
SCm=(1-(880/1000)**(lstep/Ha))
SCm=SCm*lstep/Ha #per step
#mie g parameter
g=0.71
#sun position (unused)
#sunpos=normalize(np.array([1,0,10]))
#moon position (unused)
#moonpos=normalize(np.array([-6,0.2,2]))
#sun intensity(array) (unused)
#sunI=normalize(np.array([255,242,237]))*1.6E+9
#moon intensity (array) (unused)
#moonI=normalize(np.array([100,85,73]))*2.5E+3
#sun size deg angular radius (unused)
#sunsize=0.26656
#moon size deg angular radius (unused)
#moonsize=0.26431
#samples per pixel
samples=8000
#resolution keep 90:360 ratio, whole numbers only!
res=np.array([18,72])
angscaling=360/res[1]

In [9]:
###Image Import and settings
imagein = im.open('DaytonaBeach.tif')
groundlightarray=np.array(imagein)
#decimal Latitude (insert into last value)
Lat=np.pi/180*29.21083333

dx=15/(3600*360)*2*np.pi*np.sqrt(((re**2*np.cos(Lat))**2+(rp**2*np.sin(Lat))**2)/((re*np.cos(Lat))**2+(rp*np.sin(Lat))**2))
dy=463.05

#camera position, X and Y from image coordinates (gimp displays this)
x=470
y=502

In [11]:
###main loop
t=time.time()
#create blank u*v*color image array
image=np.zeros((res[0],res[1],3))

#loop over u's (horizontal)
u=1
while u<res[1]+1:
    #loop over v's (vertical)
    v=1
    while v<res[0]+1:
        #loop over colors
        chrom=0
        while chrom<3:
            #create blank light array
            LightArray=np.zeros(samples)
            #loop over samples
            sample=0
            while sample<samples:
                #loop over sim steps (while for below conditions from end of last step)
                step=0
                intersectcheck=0
                lightdir=np.array([[np.cos(angscaling*np.pi/180*u)*np.sin(angscaling*np.pi/180*v)],[np.sin(angscaling*np.pi/180*u)*np.sin(angscaling*np.pi/180*v)],[np.cos(angscaling*np.pi/180*v)]])
                lightpos=np.zeros((3,1))
                #start individual light ray
                while step<lifespan and intersectcheck==0:
                    #check for intersections
                    if lightpos[2,0]<0:
                        #ground case
                        intersectcheck=1
                        LightArray[sample]=groundlightarray[y-round(lightpos[1,0]/dy),x+round(lightpos[0,0]/dx)]



                        
                        #if lightpos[0,0]>1000 and lightpos[0,0]<5000 and lightpos[1,0]>1000 and lightpos[1,0]<5000:
                        #    LightArray[sample]=1
                        #else:
                        #    LightArray[sample]=0
                        
                    if lightpos[2,0]>Ha:
                        #atmosphere escape case
                        LightArray[sample]=0
                        intersectcheck=1
                    #new position
                    lightpos=lightpos+lightdir*lstep
                    #check for scattering 
                    if (Rscatter(SCr,ll[chrom]))==1:
                        #rayleigh scatter case
                        #axes
                        axis1=normalize(np.array([[0],[0],[-1]])+lightdir[2]*lightdir)
                        axis2=lightdir
                        #angles
                        angle1=np.arccos(muR())
                        angle2=random.random()*2*np.pi
                        #rotate to new direction
                        lightdir=normalize(np.matmul(rotate(angle2,axis2),np.matmul(rotate(angle1,axis1),lightdir)))
                        #no scatter case
                        lightdir=lightdir
                    elif (Mscatter(SCm))==1:
                        #mie scatter case
                        #axes
                        axis1=normalize(np.array([[0],[0],[-1]])+lightdir[2]*lightdir)
                        axis2=lightdir
                        #angles
                        q=4-8*random.random()
                        a=np.sqrt(q**2+4)
                        angle1=np.arccos(muM(g))
                        angle2=random.random()*2*np.pi
                        #rotate to new direction
                        lightdir=normalize(np.matmul(rotate(angle2,axis2),np.matmul(rotate(angle1,axis1),lightdir)))
                    else:
                        #no scatter case
                        lightdir=lightdir
                    step+=1
                sample+=1
            image[v-1,u-1,chrom]=np.mean(LightArray)
            chrom+=1
        v+=1
    u+=1
print("Time elapsed: ", time.time()-t)

Time elapsed:  13875.826639652252


In [13]:
###image processing and output
#max value is set to 128 for autogain
#gain=128/np.max(image)
gain=1000
imagetyped=(image*gain).astype(np.uint8)
imageout=im.fromarray(imagetyped)
imageout.show()
imageout.save('SkyMap12.png')
print("gain: ",gain)
#convert units to Cd/m^2
image=image*0.00687696970307
np.savetxt("SkyMap12",np.sum(image,2))

gain:  1000
